# Retrieval Augmented Question & Answering with Amazon Bedrock using LangChain & Amazon OpenSearch

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

---

Previously, we used the Anthropic Claude model in Amazon Bedrock to demonstrate a basic Question Answering (QA) system, and learned the value of grounding a model with additional context before generating a response. In the previous notebook, we had to manually provide the model with relevant data and context ourselves. However, this approach is not fit for enterprise-level QA systems where there could be hundreds of thousands of large documents.

## Retrieval Augmented Generation (RAG)

We can improve upon this process by implementing an architecture called retrieval augmented generation (RAG). RAG retrieves data from outside the LLM's training data sources and augments the prompts by adding the relevant retrieved data as context. RAG extends the already powerful capabilities of LLMs to specific domains or an organization's internal knowledge base, without needing to retrain the model. It is a cost-effective approach to improving LLM output so it remains relevant, accurate, and useful in various contexts.

## Solution

In this notebook, we augment LLM responses to user queries by implementing RAG using context from external documents. First, we process documents and store these into a vector store. Next, we search the vector store using the user's question, and return relevant data as external context to the LLM. Finally, the LLM generates an answer to the user's question based on the new context provided.

We will walk through implementing the following two patterns: Question Answering (QA) and Conversational AI with conversation memory. 

Let’s break down the solution a little further. 

### Prepare documents for search
![Documents](./images/embeddings_lang.png)

First, the documents must be processed and then indexed in a vector store.
- Load the documents from our directory
- Process the documents by splitting them into smaller chunks
- Create a numerical vector representation of each chunk using an embeddings model
- Create an index using the chunks and the corresponding embeddings

### Respond to the user’s question
![Question](./images/chatbot_lang.png)

Once the vector store is indexed with documents and embeddings, we can search for text relevant to the question being asked. The relevant chunks are sent to the model as additional context, where the model will then generate the answer.
- Create an embedding of the input question
- Compare the question embedding with the embeddings in the index
- Fetch the (top N) relevant document chunks
- Add those chunks as part of the context in the prompt
- Send the prompt to the model under Amazon Bedrock
- Get the contextual answer based on the documents retrieved

Let's get started!

## Setup

Before running the rest of this notebook, you'll need to run the cells below to ensure necessary libraries are installed and to connect to Amazon Bedrock.

In [ ]:
import sys
import os
module_path = "../.."
sys.path.append(os.path.abspath(module_path))
from utils.environment_validation import validate_environment, validate_model_access
validate_environment()

In [ ]:
required_models = [
    "amazon.titan-embed-text-v1",
    "anthropic.claude-3-sonnet-20240229-v1:0",
    "anthropic.claude-3-haiku-20240307-v1:0",
]
validate_model_access(required_models)

In [ ]:
import json
import warnings
from pathlib import Path
from rich import print as rprint
warnings.filterwarnings('ignore')

from utils import bedrock

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None)
)

## Configure LangChain

LangChain provides convenient integrations with Amazon Bedrock and other services like vector stores and retrievers. We begin with instantiating the large language model (LLM) and the embeddings model. We are using Anthropic Claude models for text generation and Amazon Titan Embeddings G1 - Text for text embedding.

Note: Amazon Bedrock offers a choice of high-performing foundation models (FMs). You can replace the value for `model_id` with one of the available [model IDs](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html) as follows. Some models have different requirements for inputs such as prompt format. As of this writing, all models are supported in the US West (Oregon, us-west-2) Region. If you are using another AWS Region, check the latest [model support by AWS Region](https://docs.aws.amazon.com/bedrock/latest/userguide/models-regions.html).

```python
llm = BedrockChat(model_id="anthropic.claude-3-haiku-20240307-v1:0", ...)
```


In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain_aws.chat_models import BedrockChat
from langchain.load.dump import dumps

# Instantiate the LLM

model_id = "anthropic.claude-3-haiku-20240307-v1:0"

llm = BedrockChat(
    model_id=model_id,
    model_kwargs={"max_tokens": 500}
)

# Instantiate the Amazon Titan Embeddings G1 - Text embeddings model
bedrock_embeddings = BedrockEmbeddings(
    client=boto3_bedrock,
    model_id="amazon.titan-embed-text-v1" # change this model ID to use another embeddings model
)

## Usecase Introduction - Model Risk and Model Governance Assistant
In this notebook we will learn the application of RAG through a practical example. The use case we will be working on is a Model Risk and Model Governance Assistant. This assistant will help users understand the risks associated with deploying machine learning models in production. The assistant will provide information on the following topics:
- Model Risk Management
- Model Governance
- Regulatory Compliance
- Model Monitoring
- Model Validation
- And more

We will use some publicly available regulatory guideline documents to serve as the source for our RAG solution. You can vew the documents in the `../data/model_risk` directory.

## Data Preparation
We will load the documents with the help of [PyPDF in LangChain](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf).

We will utilize a few different techniques when loading the documents that will help improve the retrieval quality.

#### Outline based splitting
By default LangChain's `PyPDFLoader` will break each document up into pages. We could then potentially use a chunking strategy such as `RecursiveCharacterTextSplitter` to further break down the pages into smaller chunks. 
However, this could lead to suboptimal results if the most relevant information we are looking for is split across multiple pages. Instead, we will split the documents into sections based on the documents own table of contents. The implementation for this approach is provided in the rag_utils.outline_parser module [(source)](./rag_utils/outline_parser.py).
Note that this approach only works on PDFs that contain a table of contents.


#### Parent Document Retriever
After we've loaded the document as individual sections, we will further split these sections by paragraphs using the [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter/). These are the chunks that will be used for embeddings, however during retrieval we'll utilize the [ParentDocumentRetriever](https://python.langchain.com/docs/modules/data_connection/retrievers/parent_document_retriever/) to retrieve the entire section that the chunk belongs to. This is done to ensure that the context provided to the model is as complete as possible.


In [ ]:
from langchain.document_loaders import PyPDFLoader
from rag_utils.outline_parser import PyPDFOutlineParser

docs_path = Path("../data/model_risk")
doc_files = list(docs_path.glob("*.pdf"))

section_chunks = []

for doc_path in doc_files:
    loader = PyPDFLoader(file_path=doc_path.as_posix())
    loader.parser = PyPDFOutlineParser()
    sections = loader.load()
    for sec in sections:
        sec.metadata.update({"file": doc_path.name})
    
    section_chunks += sections
    

Each section chunk now contains a the contents and metadata associated with that section

In [ ]:
rprint(section_chunks[0])

Now let's test out our embedding model on a single section to see what an embedding looks like below. These embeddings could be generated for the entire corpus of documents and stored in a vector store for easy retrieval.

In [ ]:
try:
    sample_embedding = bedrock_embeddings.embed_query(section_chunks[0].page_content)
    modelId = bedrock_embeddings.model_id
    rprint("Embedding model Id :", modelId)
    rprint("Sample embedding of a document chunk: ", sample_embedding[:10])
    rprint("Size of the embedding: ", len(sample_embedding))

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubleshoot this issue please refer to the following resources.\
        \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/setting-up.html\
        \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\
        \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
              \x1b[0m")
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

## Create the vector store
In this workshop we will use the ***vector engine for Amazon OpenSearch Serverless.***

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application — without impacting data ingestion.

The code block below will provision an OpenSearch Serverless collection and create the vector store that we will use to store the embeddings.

In [ ]:
from rag_utils.oss_utils import create_oss_resources, delete_oss_resources, get_aws_auth
config_file = "oss_config.json"

host, config = create_oss_resources(config_file, replace=False)


Now we are ready to ingest the documents into the vector store. This can be done easily using the [LangChain OpenSearch integration](https://python.langchain.com/docs/integrations/vectorstores/opensearch) which takes in the embeddings model and the documents to create the entire vector store.

In [ ]:

from langchain.vectorstores import OpenSearchVectorSearch
from opensearchpy import RequestsHttpConnection

auth = get_aws_auth()


vector_db = OpenSearchVectorSearch(
    index_name=config["index_name"],
    embedding_function=bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 7200,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    engine="faiss",
)

Next we build the `ParentDocumentRetriever` combining an OpenSearch based vector store and key-value based `InMemoryStore`. The vector store will be used to find section segments that were generated using through splitting with the `RecursiveCharacterSplitter`. Each section segment will contain a key reference to the full section document. The key reference will be used to retrieve the entire section text. Note that the `InMemoryStore` is essentially a python dictionary, in production you would want to use a persistent store such as [DynamoDB](https://aws.amazon.com/dynamodb/).

In [ ]:
from langchain.storage import InMemoryStore
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
import pickle
import time


child_splitter = RecursiveCharacterTextSplitter(
    separators=["\n", "\n\n"], chunk_size=2000, chunk_overlap=250
)

in_memory_store_file = config["index_name"] + "_store.pkl"

# if we previously ingested the docs we can reuse the existing index
if Path(in_memory_store_file).exists():
    store = pickle.load(open(in_memory_store_file, "rb"))
    
    retriever = ParentDocumentRetriever(
        vectorstore=vector_db,
        docstore=store,
        child_splitter=child_splitter,
    )

# ingest the document into the index
else:
    store = InMemoryStore()
    
    retriever = ParentDocumentRetriever(
        vectorstore=vector_db,
        docstore=store,
        child_splitter=child_splitter,
    )
    
    retriever.add_documents(section_chunks, ids=None)
    pickle.dump(store, open(in_memory_store_file, "wb"))

## Searching the vector store
Before we get into the parent document retrieval, let's first explore the various ways that we can query the vector store exclusively.

### Semantic search methods
[Semantic search](https://opensearch.org/docs/latest/search-plugins/semantic-search/) considers the context and intent of a query. In OpenSearch, semantic search is facilitated by neural search with text embedding models. Semantic search creates a dense vector (a list of floats) and ingests data into a k-NN index.

Short for k-nearest neighbors, the k-NN plugin enables users to search for the k-nearest neighbors to a query point across an index of vectors.  To determine the neighbors, you can specify the space (the distance function) you want to use to measure the distance between points. We will explore some of the distance functions later in this lab.

The k-NN plugin supports three different methods for obtaining the k-nearest neighbors from an index of vectors:
- Approximate k-NN
- Script Score k-NN
- Painless extensions

#### Approximate k-NN search
Standard k-NN search methods compute similarity using a brute-force approach that measures the nearest distance between a query and a number of points, which produces exact results. This works well in many applications. However, in the case of extremely large datasets with high dimensionality, this creates a scaling problem that reduces the efficiency of the search. [Approximate k-NN search](https://opensearch.org/docs/latest/search-plugins/knn/approximate-knn/) methods can overcome this by employing tools that restructure indexes more efficiently and reduce the dimensionality of searchable vectors. Using this approach requires a sacrifice in accuracy but increases search processing speeds appreciably.

Of the three search methods the k-NN plugin provides, this method offers the best search scalability for large datasets. This approach is the preferred method when a dataset reaches hundreds of thousands of vectors.

In [ ]:
# Search query
query = "What can be considered a model?"

# Search for the 3 most relevant documents
results = vector_db.similarity_search(query, k=3)

# sometimes it takes a few seconds for the vector store to be ready
while len(results) == 0:
    print("Vector store not yet ready")
    results = vector_db.similarity_search(query, k=3)
    time.sleep(5)


rprint(dumps(results, pretty=True))

#### Exact k-NN with scoring script
The k-NN plugin implements the OpenSearch [score script](https://opensearch.org/docs/latest/search-plugins/knn/knn-score-script/) plugin that you can use to find the exact k-nearest neighbors to a given query point.

Because the score script approach executes a brute force search, it doesn’t scale as well as the approximate approach. This approach is preferred for searches over smaller bodies of documents or when a pre-filter is needed. Using this approach on large indexes may lead to high latencies.

In [ ]:
results = vector_db.similarity_search(
    query, 
    k=2,
    is_appx_search=False,
    search_type="script_scoring"
)

rprint(dumps(results, pretty=True))

#### k-NN Painless Scripting extensions (exact)
Currently, the vector engine for Amazon OpenSearch serverless supports the approximate k-NN search and scoring script search methods. Below is an example of the [painless scripting](https://opensearch.org/docs/latest/search-plugins/knn/painless-functions/) search method on an Amazon OpenSearch service for your reference.

Similar to the k-NN Script Score, you can use this method to perform a brute force, exact k-NN search across an index. This approach has slightly slower query performance compared to the k-NN scoring script. If your use case requires more customization over the final score, you should use this approach over k-NN scoring script.

```python
results = docsearch.similarity_search(
    query, 
    is_appx_search=False,
    search_type="painless_scripting"
)

print(dumps(results, pretty=True))
```

#### Exact k-NN search with filters
You can apply [k-NN search with filters](https://opensearch.org/docs/latest/search-plugins/knn/filter-search-knn/) with either the  scoring script or painless extension search methods. Filters can greatly reduce the number of vectors to be searched. Using the k-NN score script, you can apply a filter on an index before executing the nearest neighbor search (sometimes referred to as a pre-filter search). This is useful for dynamic search cases where the index body may vary based on other conditions.

In [ ]:
query = "What are the acceptable model evaluation techniques?"

# filter on a specific document
pre_filter = {"bool": {"filter": {"match": {"metadata.file": "sr1107a1.pdf"}}}}

# Pre-filter results
results = vector_db.similarity_search(
    query, 
    k=2,
    is_appx_search=False,
    search_type="script_scoring",
    pre_filter=pre_filter   
)

rprint(dumps(results, pretty=True))

#### Spaces - similarity or distance measures

Approximate Search through OpenSearch supports the following similarity or distance measures:

**Cosine similarity** – The cosine of the angle between two vectors in a vector space.

**Euclidean distance** – The straight-line distance between points.

**L1 (Manhattan) distance** – The sum of the differences of all of the vector components. L1 distance measures how many orthogonal city blocks you need to traverse from point A to point B.

**L-infinity (chessboard) distance** – The number of moves a King would make on an n-dimensional chessboard. It’s different than Euclidean distance on the diagonals—a diagonal step on a 2-dimensional chessboard is 1.41 Euclidean units away, but 2 L-infinity units away.

**Inner product** – The product of the magnitudes of two vectors and the cosine of the angle between them. Usually used for natural language processing (NLP) vector similarity.

We can specify the distance measure in the `space_type` parameter when we load our documents as seen below.

```python
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss",
    space_type="l2", # Options are: “l2”, “l1”, “cosinesimil”, “linf”, “innerproduct”; default: “l2”
)
```

#### [Optional] Engines and algorithms
The Approximate k-NN search methods leveraged by OpenSearch use approximate nearest neighbor (ANN) algorithms from the [NMSLIB](https://github.com/nmslib/nmslib), [FAISS](https://github.com/facebookresearch/faiss), and [Lucene](https://lucene.apache.org/) libraries to power k-NN search.

The engine details are as follows:

- Non-Metric Space Library (NMSLIB) – NMSLIB implements the HNSW ANN algorithm
- Facebook AI Similarity Search (FAISS) – FAISS implements both HNSW and IVF ANN algorithms
- Lucene – Lucene implements the HNSW algorithm

Each of the three engines used for approximate k-NN search has its own attributes that make one more sensible to use than the others in a given situation. In general, NMSLIB and FAISS should be selected for large-scale use cases. Lucene is a good option for smaller deployments, but offers benefits like smart filtering where the optimal filtering strategy—pre-filtering, post-filtering, or exact k-NN—is automatically applied depending on the situation.

We can specify the engine as shown below.

Note: As of this writing, Amazon OpenSearch Serverless vector search collections don't support the Apache Lucene ANN engine. Vector search collections only support the HNSW algorithm with FAISS and do not support IVF and IVFQ. Please check the updated [limitations](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/serverless-vector-search.html#serverless-vector-limitations). 

```python
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss", # Options are: “nmslib”, “faiss”, “lucene”; default: “nmslib”
)
```

**For HNSW, we can tune the m, ef_construction, and ef_search parameters to achieve our desired trade-off:**

**m** – Controls the maximum number of edges a node in a graph can have. Because each node has to store all of its edges, increasing this value will increase the memory footprint, but also increase the connectivity of the graph, which will improve recall.

**ef_construction** – Controls the size of the candidate queue for edges when adding a node to the graph. Increasing this value will increase the number of candidates to consider, which will increase the index latency. However, because more candidates will be considered, the quality of the graph will be better, leading to better recall during search.

**ef_search** – Similar to ef_construction, it controls the size of the candidate queue for graph traversal during search. Increasing this value will increase the search latency, but will also improve the recall.

In general, we chose configurations that gradually increase the parameters, as detailed in the following table.

![](./images/hnsw_parameters.png)

Below is an example using the **FAISS** engine and providing a parameter configuration that provides a balance between latency, memory, and recall (see table above).

```python
docsearch = OpenSearchVectorSearch.from_documents(
    docs,
    bedrock_embeddings,
    opensearch_url=host,
    http_auth=auth,
    timeout = 100,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection,
    index_name=index_name,
    engine="faiss",
    space_type="l2",
    m=16, # maximum number of edges
    ef_construction=128, # size of the candidate queue for edges
    ef_search=128 # size of the candidate queue for graph traversal
)
```

### Maximum marginal relevance search (MMR)
If you’d like to look up for some similar documents, but you’d also like to receive diverse results, MMR is a method you should consider. Maximal marginal relevance optimizes for similarity to query AND diversity among selected documents. It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

In [ ]:
results = vector_db.max_marginal_relevance_search(
    query, 
    k=2,
    fetch_k=10,
    lambda_param=0.5
)

rprint(dumps(results, pretty=True))


## Parent Document Retrieval
The queries above returned just the matching segments from the vector store. Now let's see what happens when we invoke the `ParentDocumentRetriever` that was defined earlier.

In [ ]:
query = "What are the acceptable model evaluation techniques?"

# search with just the vector store
section_segments = vector_db.similarity_search(query, k=10)

retriever.search_kwargs = {"k": 10}

# search with the parent retriever
full_sections = retriever.invoke("What are the acceptable model evaluation techniques?")

rprint(f"Vector search returned {len(section_segments)} segments while the parent retriever returned {len(full_sections)} sections")

In the above example, we set k=10 which should return 10 matches from the vector store. However the parent document retriever should return fewer documents as it will return the entire section that multiple returned chunks can belong to. 

## Orchestrating RAG using LangChain
Now that we can query our vector database for documents, we can retrieve data from outside of a large language model's training data sources and augment our prompts by adding the relevant retrieved data in context.

We can use LangChain to build applications that read data from stored internal documents and summarize them into conversational responses. We can create a Retrieval Augmented Generation (RAG) workflow that introduces new information to the language model during prompting. Implementing context-aware workflows like RAG reduces model hallucination and improves response accuracy.

### Single turn generative question answering

Let's start with a simple example where given a user query we retrieve relevant documents from the vector store and use the retrieved documents as context to generate a response.

We'll construct a prompt template that will take the user's question and the retrieved documents as context and generate a response.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

template = """Answer the question based only on the following context. 
If the context does not provide sufficient information to answer the question, politely indicate that you are unable to assist. 
Only answer questions related to model risk and model governance.

<context>
{context}
</context>

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

# in the first step we retrieve the context and pass through the input question
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
    
)

# In the subsequent steps pass the context and question to the prompt, send the prompt to the llm and parse the output as a string
chain = setup_and_retrieval | prompt | llm | output_parser

Let’s try this with our earlier query:

In [ ]:
query = "What can be considered a model?"

response = chain .invoke(query)
rprint(response)

In [ ]:
query = "What are some acceptable model evaluation techniques?"

response = chain.invoke(query)
rprint(response)

## Improving our solution

The above solution works but is notably missing a number of key features including:
- Ability to have multi-turn conversations
- Ability to return source documents
- The response are constrained only to what is in the documents which may limit the usefulness of the tool

In the following section we will address these gaps and make further enhancements such as utilizing some of the prompting best practices for Claude, and refining the prompt a bit to provide more natural responses.


#### Set up our prompt templates
The updated solution will utlize 3 prompt templates including:
- **Condense template** that will look at the conversation history and generate a standalone response. This is required as a user's follow up question may not include sufficient context to perform an effective retrieval search. We'd therefore need to ask the model to rephrase the question in such a way that all the necessary context is included.
- **Document template** that will format the documents using Claude best practices (i.e. xml tags) before feeding them into the answer template 
- **Answer template** that will take the user's question and the retrieved documents and generate a response.


In [ ]:
from operator import itemgetter
from langchain.schema import format_document
from langchain_core.messages import  get_buffer_string
from langchain_core.runnables import  RunnableLambda
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate

In [ ]:
# template to rephrase the question
condense_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
Skip any preamble or summarization, simply generate the rephrased question.
<history>
{chat_history}
</history>
Follow Up Input: {question}
"""

CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(condense_template)

In [ ]:
# template to generate a response
answer_template = """You are a foremost expert in model risk and model governance. Your job is to advise users on the best practices and guidelines in these areas.
The context below provides relevant information to answer the question. You must use this context to provide a detailed and accurate response.
Only answer questions related to model risk and model governance, if a user asks a question about a different topic, politely decline.

<context>
{context}
</context>

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(answer_template)

In [ ]:
# template and function to format context documents in xml tags

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="<document section_title={section_title}>{page_content}</document>")

def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

Next we need a place to store the conversation state. Here we'll use the in-memory [ConversationBufferMemory](https://python.langchain.com/docs/modules/memory/types/buffer/) to store the conversation history. This will allow us to keep track of the conversation history and use it to generate a standalone response when needed. In practice, you'd likely use something like [DynamoDB](https://python.langchain.com/docs/integrations/memory/aws_dynamodb/) to store the conversation history.


In [ ]:
# instantiate a blank memory buffer
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question"
)

# First we add a step to load memory from the buffer to feed into the prompt
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
)

# Next we generate the standalone question
standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: get_buffer_string(
            x["chat_history"], human_prefix="human", ai_prefix="assistant"
        ),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}
# Retrieve the documents using the generated question
retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}
# Construct the inputs for the final prompt with the formatted context docs
final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}
# Send the final prompt to the llm
answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}
# And now we put it all together!
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [ ]:
inputs = {"question": "What types of model risks should be documented?"}
result = final_chain.invoke(inputs)
rprint(result["answer"].content)
rprint("source documents:\n", json.dumps([doc.metadata for doc in  result["docs"]], indent=2))

# after every conversation turn we update the conversation state in the memory buffer
memory.save_context(inputs, {"answer": result["answer"].content})

In [ ]:
inputs = {"question": "Can you provide some examples for a fraud detection model?"}
result = final_chain.invoke(inputs)
rprint(result["answer"].content)
rprint("source documents:\n", json.dumps([doc.metadata for doc in  result["docs"]], indent=2))

memory.save_context(inputs, {"answer": result["answer"].content})

In [ ]:
inputs = {"question": "How about for a credit scoring model?"}
result = final_chain.invoke(inputs)
print(result["answer"].content)
print("source documents:\n", json.dumps([doc.metadata for doc in  result["docs"]], indent=2))

memory.save_context(inputs, {"answer": result["answer"].content})

### Clean up
You have reached the end of this workshop. Following cell will delete all created resources.


In [ ]:
delete_oss_resources(config_file)

## Conclusion
In the above implementation of RAG based Question Answering and Conversational AI, we have explored the following concepts and how to implement them using the LangChain integrations for Amazon Bedrock and Amazon OpenSearch Serverless:

- Loading documents and processing them into smaller chunks
- Creating a vector store using vector engine for Amazon OpenSearch Serverless
- Generating embeddings with an embeddings model
- Searching the vector store to retrieve context relevant to the question
- Performing Generative Question Answering using foundation models
- Improving trust in our system by providing citations with every answer
- Preparing prompt templates to use as input to the LLM
- Storing conversation memory and providing the history as context to the LLM

### Next steps
- Experiment with different vector stores
- Leverage various text and embedding models available through Amazon Bedrock to see alternate outputs
- Explore options such as persistent storage of embeddings and document chunks
- Use Amazon Bedrock Knowledge Bases, a fully managed RAG capability with built-in session context management

# Thank You